## Query existing database

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
print(pd.__version__)

1.1.3


In [3]:
database_path = 'Resources/dow.sqlite'
engine = create_engine(f'sqlite:///{database_path}')

In [4]:
# method 1
data = engine.execute("SELECT * FROM dow")
print(type(data))

i = 0
for r in data:
    if i < 5:
        print(r)
        i += 1

<class 'sqlalchemy.engine.result.ResultProxy'>
(1, 1, 'AA', '2011-01-07', 15.82, 16.72, 15.78, 16.42, 239655616, None)
(2, 1, 'AA', '2011-01-14', 16.71, 16.71, 15.64, 15.97, 242963398, None)
(3, 1, 'AA', '2011-01-21', 16.19, 16.38, 15.6, 15.79, 138428495, None)
(4, 1, 'AA', '2011-01-28', 15.87, 16.63, 15.82, 16.13, 151379173, None)
(5, 1, 'AA', '2011-02-04', 16.18, 17.39, 16.18, 17.14, 154387761, None)


In [5]:
# method 2
conn = engine.connect()
df = pd.read_sql("SELECT * FROM dow", conn)
df.head()

,id,quarter,stock,date,open_price,high_price,low_price,close_price,volume,percent_change
0,1,1,AA,2011-01-07,15.82,16.72,15.78,16.42,239655616,None
1,2,1,AA,2011-01-14,16.71,16.71,15.64,15.97,242963398,None
2,3,1,AA,2011-01-21,16.19,16.38,15.60,15.79,138428495,None
3,4,1,AA,2011-01-28,15.87,16.63,15.82,16.13,151379173,None
4,5,1,AA,2011-02-04,16.18,17.39,16.18,17.14,154387761,None


<br>

## Create table

##### SQLAlchemy uses Python classes as its primary means to communicate and make changes to SQL databases
##### This makes SQLAlchemy an ORM because it uses objects to map changes to SQL tables/databases

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

In [7]:
# Create Pet closs (table)
class Pet(Base):
    __tablename__ = "pet"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    color = Column(String(255))
    kind = Column(String(255))
    age = Column(Integer)

In [8]:
# Create Robot class (table)
class Robot(Base):
    __tablename__ = "robot"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    origin = Column(String(255))


In [9]:
# Create database connection
database_path = 'Resources/pets_cafe.db'
engine = create_engine(f'sqlite:///{database_path}')


In [10]:
# Drop Table if exists
if os.path.exists(database_path):
    Base.metadata.drop_all(engine)

# Create Pet table and Robot table in the database
Base.metadata.create_all(engine)


In [11]:
# Create instances for each class
rabbit = Pet(name='mimi', color='white', kind='rabbit', age=2)
rabbit2 = Pet(name='lucy', color='pink', kind='rabbit', age=1)
cat = Pet(name='sandy', color='white', kind='cat', age=6)
cat2 = Pet(name='poker', color='black', kind='cat', age=3)
cat3  = Pet(name='sunshine', color='white', kind='cat', age=2)
owl = Pet(name='cookie', color='brown', kind='owl', age=4)
robot = Robot(name='guy', origin='Spain')

In [12]:
# Create session
session = Session(bind=engine)

In [13]:
# Add objects
# Queue up the queries
session.add(rabbit)
session.add(rabbit2)
session.add(cat)
session.add(cat2)
session.add(cat3)
session.add(owl)
session.add(robot)

In [14]:
# Check queue and see which data is ready to go into the database
session.new

IdentitySet([<__main__.Pet object at 0x0000024C6B323D88>, <__main__.Pet object at 0x0000024C6ED8C388>, <__main__.Pet object at 0x0000024C6ED8CCC8>, <__main__.Pet object at 0x0000024C6ED8CA08>, <__main__.Pet object at 0x0000024C6ED8CDC8>, <__main__.Pet object at 0x0000024C6ED8CE48>, <__main__.Robot object at 0x0000024C6ED8CEC8>])

In [15]:
# commit() flushes whatever remaining changes remain to the database
# and commits the transaction
session.commit()


In [16]:
# Check queue again
session.new

IdentitySet([])

In [17]:
# Check tables
Base.metadata.tables

immutabledict({'pet': Table('pet', MetaData(bind=None), Column('id', Integer(), table=<pet>, primary_key=True, nullable=False), Column('name', String(length=255), table=<pet>), Column('color', String(length=255), table=<pet>), Column('kind', String(length=255), table=<pet>), Column('age', Integer(), table=<pet>), schema=None), 'robot': Table('robot', MetaData(bind=None), Column('id', Integer(), table=<robot>, primary_key=True, nullable=False), Column('name', String(length=255), table=<robot>), Column('origin', String(length=255), table=<robot>), schema=None)})

<br>

## Query table

In [18]:
# Query all items
# method 1
pets = session.query(Pet)
for p in pets:
    print(p.name)

mimi
lucy
sandy
poker
sunshine
cookie


In [19]:
# method 2
session.query(Pet.name, Pet.kind, Pet.color, Pet.age).all()

[('mimi', 'rabbit', 'white', 2),
 ('lucy', 'rabbit', 'pink', 1),
 ('sandy', 'cat', 'white', 6),
 ('poker', 'cat', 'black', 3),
 ('sunshine', 'cat', 'white', 2),
 ('cookie', 'owl', 'brown', 4)]

In [20]:
cat_count = session.query(Pet).filter_by(kind='cat').count()
cat_count

3

In [21]:
# More than 1 filter condition

# method 1
white_cat = session.query(Pet).filter_by(kind='cat', color='white')
white_cat.count()

2

In [22]:
# method 2
white_cat = session.query(Pet.name, Pet.color).filter((Pet.kind=='cat') & (Pet.color=='white'))
white_cat.count()

2

In [23]:
type(white_cat)

sqlalchemy.orm.query.Query

In [24]:
for row in white_cat.all():
    print(row)

('sandy', 'white')
('sunshine', 'white')


In [25]:
# How many types of pet
by_kind = session.query(Pet).group_by(Pet.kind)
by_kind.count()

3

## Update data

In [26]:
older_owl = session.query(Pet).filter_by(kind='owl').first()
print(older_owl.age)
older_owl.age += 1
print(older_owl.age)


4
5


In [27]:
# Check queue and see which data is ready to be updated
session.dirty

IdentitySet([<__main__.Pet object at 0x0000024C6ED8CE48>])

In [28]:
session.commit()

In [29]:
session.dirty

IdentitySet([])

In [30]:
session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('sandy', 'white', 'cat', 6),
 ('poker', 'black', 'cat', 3),
 ('sunshine', 'white', 'cat', 2),
 ('cookie', 'brown', 'owl', 5)]

<br>

<br>

## Delete Data

In [31]:
# Delete one black cat
black_cat = session.query(Pet).filter_by(kind='cat', color='black').one()
black_cat

In [32]:
session.delete(black_cat)

In [33]:
# Check queue and see which data is ready to be deleted
session.deleted

IdentitySet([<__main__.Pet object at 0x0000024C6ED8CA08>])

In [34]:
session.commit()

In [35]:
session.deleted

IdentitySet([])

In [36]:
session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('sandy', 'white', 'cat', 6),
 ('sunshine', 'white', 'cat', 2),
 ('cookie', 'brown', 'owl', 5)]

In [37]:
# Delete all white cats
session.query(Pet).filter_by(kind='cat', color='white').delete()
session.commit()


session.query(Pet.name, Pet.color, Pet.kind, Pet.age).all()

[('mimi', 'white', 'rabbit', 2),
 ('lucy', 'pink', 'rabbit', 1),
 ('cookie', 'brown', 'owl', 5)]

In [38]:
youngest = session.query(Pet).order_by(Pet.age).first()
youngest.name

'lucy'

In [39]:
# Close session
session.close()

<br>

## Reflection of existing database

#### To automatically create Object Relationship Mapping classes from an exisiting databse

In [40]:
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [41]:
engine = create_engine(f'sqlite:///Resources/dow.sqlite')
Base = automap_base()

# Use Base class to reflect database tables
Base.prepare(engine, reflect=True)

# Print all classes (tables) mapped to Base
Base.classes.keys()

['dow']

In [42]:
# Assign dow class to Dow
Dow = Base.classes.dow

In [43]:
session = Session(engine)

first_dow = session.query(Dow).first()
first_dow

In [44]:
# Show columns and data in dictionary format
first_dow.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x24c6ee10f48>,
 'quarter': 1,
 'date': '2011-01-07',
 'high_price': 16.72,
 'close_price': 16.42,
 'percent_change': None,
 'stock': 'AA',
 'id': 1,
 'open_price': 15.82,
 'low_price': 15.78,
 'volume': 239655616}

<br>

#### Query Dow Table

In [45]:
# First 5 close prices only
session.query(Dow.stock, Dow.close_price).limit(5).all()

[('AA', 16.42), ('AA', 15.97), ('AA', 15.79), ('AA', 16.13), ('AA', 17.14)]

In [46]:
# First 5 close prices only
for row in session.query(Dow.stock, Dow.close_price).limit(5).all():
    print(row)

('AA', 16.42)
('AA', 15.97)
('AA', 15.79)
('AA', 16.13)
('AA', 17.14)


In [47]:
# Total number of dates
session.query(func.count(Dow.date)).all()

[(750)]

In [48]:
# The last date in the data
session.query(Dow.date).order_by(Dow.date.desc()).first()

('2011-06-24')

In [49]:
# Number of dates that are greater than June 1 2011
len(session.query(Dow.date).filter(Dow.date > '2011-06-01').\
                        order_by(Dow.date).all())

120

<br>

#### Use datetime format in SQLAlchemy (Sqlite doesn't support a data column type)

In [50]:
import datetime as dt

dt.date.today()

datetime.date(2020, 12, 6)

In [51]:
print(dt.date.today())
print(dt.datetime(2020,12,5))

# 1 week before today
print(dt.date.today() - dt.timedelta(weeks=1))

# Parse month from date
print(dt.date.today().strftime('%m'))

2020-12-06
2020-12-05 00:00:00
2020-11-29
12


In [52]:
june = '06'
len(session.query(Dow.date).filter(func.strftime('%m', Dow.date) == june).all())

120

In [53]:
session.close()

<br>

## Read query into Pandas DataFrame

In [54]:
# method 1: query-> top10 -> list-> df

# Top 10 stocks with highest volume
top10vol = session.query(Dow.stock, Dow.volume).order_by(Dow.volume.desc()).all()[:10]
top10vol

[('BAC', 1453438639),
 ('BAC', 1054415375),
 ('CSCO', 1000362015),
 ('BAC', 982445809),
 ('BAC', 889460755),
 ('BAC', 873241317),
 ('BAC', 805210688),
 ('BAC', 792380862),
 ('BAC', 760935694),
 ('BAC', 757192224)]

In [55]:
# unpack tuples using list comprehension
tickers = [t[0] for t in top10vol]
print(tickers)
vols = [int(t[1]) for t in top10vol]
print(vols)

['BAC', 'BAC', 'CSCO', 'BAC', 'BAC', 'BAC', 'BAC', 'BAC', 'BAC', 'BAC']
[1453438639, 1054415375, 1000362015, 982445809, 889460755, 873241317, 805210688, 792380862, 760935694, 757192224]


In [56]:
# Put into dataframe
df = pd.DataFrame(top10vol, columns=['tic', 'vol'])
df

,tic,vol
0,BAC,1453438639
1,BAC,1054415375
2,CSCO,1000362015
3,BAC,982445809
4,BAC,889460755
5,BAC,873241317
6,BAC,805210688
7,BAC,792380862
8,BAC,760935694
9,BAC,757192224


In [57]:
# Reverse order in df
df.iloc[::-1][:5]

,tic,vol
9,BAC,757192224
8,BAC,760935694
7,BAC,792380862
6,BAC,805210688
5,BAC,873241317


In [58]:
# method 2: query-> df -> top10

stmt = session.query(Dow.stock, Dow.volume).order_by(Dow.volume.desc()).statement
df2 = pd.read_sql_query(stmt, session.bind)

session.close()
df2.head(10)

,stock,volume
0,BAC,1453438639
1,BAC,1054415375
2,CSCO,1000362015
3,BAC,982445809
4,BAC,889460755
5,BAC,873241317
6,BAC,805210688
7,BAC,792380862
8,BAC,760935694
9,BAC,757192224


<br>

## Inspector Tool

In [59]:
# Create inspector object
inspector = inspect(engine)

# Get the table names from a database
inspector.get_table_names()

['dow']

In [60]:
# print column names and types
cols = inspector.get_columns('dow')
for col in cols:
    print(col['name'], col['type'])

id INTEGER
quarter INTEGER
stock TEXT
date TEXT
open_price FLOAT
high_price FLOAT
low_price FLOAT
close_price FLOAT
volume INTEGER
percent_change FLOAT


<br>

## Join Tables

In [61]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

class Continent(Base):
    __tablename__ = "continent"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    country = Column(String)

class Country(Base):
    __tablename__ = "country"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    capital = Column(String)

database_path = 'Resources/geo.sqlite'
engine = create_engine(f'sqlite:///{database_path}')

if os.path.exists(database_path):
    Base.metadata.drop_all(engine)
    
# Create Continent and Country table in the database
Base.metadata.create_all(engine)

# Create sessions
session = Session(bind=engine)

# Add data
session.add(Continent(name='Europe', country='Spain'))
session.add(Continent(name='Europe', country='Italy'))
session.add(Continent(name='Asia', country='Japan'))
session.add(Continent(name='Africa', country='Egypt'))
session.add(Country(name='Spain', capital='Madrid'))
session.add(Country(name='Italy', capital='Rome'))
session.add(Country(name='Japan', capital='Tokyo'))
session.add(Country(name='Egypt', capital='Cairo'))
session.commit()


In [62]:
# Join tables method 1:
join1 = session.query(Continent, Country).filter(Continent.country == Country.name).limit(5).all()
for row in join1:
    continent, country = row
    print(continent.name, continent.country, country.capital)
    
session.close()

print('-' * 15)

# Join tables method 2:
cols = [Continent.name, Continent.country, Country.capital]
join2 = session.query(*cols).filter(Continent.country == Country.name).limit(5).all()
for data in join2:
    continent, country, capital = data
    print(f'{country} is in {continent} and its captial is {capital}')

session.close()

Europe Spain Madrid
Europe Italy Rome
Asia Japan Tokyo
Africa Egypt Cairo
---------------
Spain is in Europe and its captial is Madrid
Italy is in Europe and its captial is Rome
Japan is in Asia and its captial is Tokyo
Egypt is in Africa and its captial is Cairo
